In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from itertools import product
import warnings

In [2]:
import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2

from modules.config import *
from modules.storage import get_results_df, get_availability_model_data
from modules.neural_network import execute_stage, get_first_stage_hyperparameters,get_second_stage_hyperparameters, get_third_stage_hyperparameters

In [3]:
def get_model_data(h3_res, time_interval_length):
    model_data_train, model_data_test = get_availability_model_data(
        h3_res, time_interval_length
    )
    # return model_data_train.sample(frac=0.1), model_data_test
    return model_data_train, model_data_test


In [4]:
execute_stage(
    get_model_data,
    NN_FIRST_STAGE_AVAILABILITY_RESULTS_PATH,
    get_first_stage_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)

  0%|          | 0/5 [00:00<?, ?it/s]

[10:26:13] batch_size: 512 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[10:26:13] batch_size: 256 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[10:26:13] batch_size: 128 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[10:26:13] batch_size: 64 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[10:26:13] batch_size: 32 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained


In [5]:
results = get_results_df(NN_FIRST_STAGE_AVAILABILITY_RESULTS_PATH)

best_batch_size = (
    results[
        (results["h3_res"] == TUNE_H3_RESOLUTION)
        & (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
    ]
    .sort_values(by="val_mse", ascending=True)["batch_size"]
    .iloc[0]
)

first_stage_hyperparameters = get_first_stage_hyperparameters()
batch_sizes = list(map(lambda x: x['batch_size'], first_stage_hyperparameters))
max_batch_size = max(batch_sizes)
min_batch_size = min(batch_sizes)

print(f"best batch_size: **{best_batch_size}** - min: {min_batch_size} - max: {max_batch_size}")


best batch_size: **64** - min: 32 - max: 512


In [6]:
get_hyperparameters = lambda : get_second_stage_hyperparameters(best_batch_size)
execute_stage(
    get_model_data,
    NN_SECOND_STAGE_AVAILABILITY_RESULTS_PATH,
    get_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)

  0%|          | 0/18 [00:00<?, ?it/s]

[10:26:14] batch_size: 64 - nodes_per_feature: 0.5 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 0.5 - n_layers: 1 - activation: tanh - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 0.5 - n_layers: 2 - activation: relu - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 0.5 - n_layers: 2 - activation: tanh - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 0.5 - n_layers: 3 - activation: relu - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 0.5 - n_layers: 3 - activation: tanh - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 1 - n_layers: 1 - activation: relu - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 1 - n_layers: 1 - activation: tanh - dropout: -1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 1 - n_layers: 2 - activation:

In [7]:
results = get_results_df(NN_SECOND_STAGE_AVAILABILITY_RESULTS_PATH)
best_model = (
    results[
        (results["h3_res"] == TUNE_H3_RESOLUTION)
        & (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
    ]
    .sort_values(by="val_mse", ascending=True)
    .iloc[0]
)

best_config = {
    "batch_size": best_model["batch_size"],
    "nodes_per_feature": best_model["nodes_per_feature"],
    "n_layers": best_model["n_layers"],
    "activation": best_model["activation"],
}

best_config


{'batch_size': 64,
 'nodes_per_feature': 1.5,
 'n_layers': 3,
 'activation': 'tanh'}

In [8]:
get_hyperparameters = lambda : get_third_stage_hyperparameters(
    best_batch_size=best_model["batch_size"],
    best_nodes_per_feature=best_model["nodes_per_feature"],
    best_n_layers=best_model["n_layers"],
    best_activation=best_model["activation"],
)
execute_stage(
    get_model_data,
    NN_THIRD_STAGE_AVAILABILITY_RESULTS_PATH,
    get_hyperparameters,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
)


  0%|          | 0/5 [00:00<?, ?it/s]

[10:26:14] batch_size: 64 - nodes_per_feature: 1.5 - n_layers: 3 - activation: tanh - dropout: 0 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 1.5 - n_layers: 3 - activation: tanh - dropout: 0.05 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 1.5 - n_layers: 3 - activation: tanh - dropout: 0.1 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 1.5 - n_layers: 3 - activation: tanh - dropout: 0.2 # already trained
[10:26:14] batch_size: 64 - nodes_per_feature: 1.5 - n_layers: 3 - activation: tanh - dropout: 0.5 # already trained


In [9]:
results = get_results_df(NN_THIRD_STAGE_AVAILABILITY_RESULTS_PATH)
best_dropout = (
	results[
		(results["h3_res"] == TUNE_H3_RESOLUTION)
		& (results["time_interval_length"] == TUNE_TIME_INTERVAL_LENGTH)
	]
	.sort_values(by="val_mse", ascending=True)["dropout"]
	.iloc[0]
)
best_config = {
	**best_config,
	"dropout": best_dropout,
}
best_config
		

{'batch_size': 64,
 'nodes_per_feature': 1.5,
 'n_layers': 3,
 'activation': 'tanh',
 'dropout': 0.1}

In [10]:
def reduce_model_data_func(get_model_data: callable, frac: float):
	def new_get_model_data(h3_res, time_interval_length):
		model_data = get_model_data(h3_res, time_interval_length)
		return model_data.sample(frac=frac)
	
	return new_get_model_data

In [11]:
list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24)]

In [12]:
best_config

{'batch_size': 64,
 'nodes_per_feature': 1.5,
 'n_layers': 3,
 'activation': 'tanh',
 'dropout': 0.1}

In [13]:
modified_best_config = {
	**best_config,
	'n_layers': 2,
	'nodes_per_feature': 0.5,
}

In [14]:
res_tuples = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS)) + ADDITIONAL_PREDICTIVE_RESOLUTIONS
res_tuples

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [15]:
gpu_overflow_res_tuples = []
# gpu_overflow_res_tuples = [(8,1), (8,2)]

In [16]:
for h3_res, time_interval_length in tqdm(res_tuples):
    tqdm.write(f"h3_res: {h3_res}, time_interval_length: {time_interval_length}", end="\r")
    current_best_config = (
        modified_best_config
        if (h3_res, time_interval_length) in gpu_overflow_res_tuples
        else best_config
    )
    execute_stage(
        get_model_data,
        NN_FOURTH_STAGE_AVAILABILITY_RESULTS_PATH,
        lambda: [current_best_config],
        h3_res,
        time_interval_length,
        test_phase=True,
        silent=True,
    )
    tqdm.write(f"h3_res: {h3_res}, time_interval_length: {time_interval_length} done")


  0%|          | 0/9 [00:00<?, ?it/s]

h3_res: 7, time_interval_length: 1 done
h3_res: 7, time_interval_length: 2 done
h3_res: 7, time_interval_length: 6 done
h3_res: 7, time_interval_length: 24 done
h3_res: 8, time_interval_length: 1 done


2022-07-08 10:26:20.239513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 10:26:20.243923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 10:26:20.244605: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 10:26:20.245538: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

h3_res: 8, time_interval_length: 2 done
h3_res: 8, time_interval_length: 6 done
h3_res: 8, time_interval_length: 24 done
h3_res: 9, time_interval_length: 24 done


In [17]:
results = get_results_df(NN_FOURTH_STAGE_AVAILABILITY_RESULTS_PATH)
results

,h3_res,time_interval_length,batch_size,nodes_per_feature,n_layers,activation,dropout,train_duration,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy
0,7,1,64,1.5,3,tanh,0.1,855.901519,45.103438,6.715909,2.734877,0.450544,0.855141
1,7,2,64,1.5,3,tanh,0.1,560.716685,47.166135,6.867761,2.816211,0.442670,0.853768
2,7,6,64,1.5,3,tanh,0.1,236.145673,53.590551,7.320557,3.066465,0.427757,0.846844
3,7,24,64,1.5,3,tanh,0.1,94.846507,93.348040,9.661679,3.883434,0.341454,0.849619
4,8,1,64,1.5,3,tanh,0.1,3703.945640,5.674175,2.382053,0.983003,0.596280,0.865919
5,8,2,64,1.5,3,tanh,0.1,1830.123308,5.697243,2.386890,0.961181,0.589726,0.862052
6,8,6,64,1.5,3,tanh,0.1,641.114363,6.773018,2.602502,1.063654,0.559425,0.853029
7,8,24,64,1.5,3,tanh,0.1,230.024005,9.372776,3.061499,1.280754,0.477692,0.889425
8,9,24,64,1.5,3,tanh,0.1,498.783041,1.812260,1.346202,0.646167,0.566795,0.822044


In [18]:
def get_availability_stats(row):
    h3_res, time_interval_length = row["h3_res"], row["time_interval_length"]

    model_data_tuple = get_model_data(h3_res, time_interval_length)
    model_data = pd.concat(model_data_tuple)
    median = model_data.outcome.median()
    mean = model_data.outcome.mean()
    std = model_data.outcome.std()
    maximum = model_data.outcome.max()
    perc_0 = model_data.outcome[model_data.outcome == 0].count() / len(model_data)

    return pd.Series(
        {
            "median": median,
            "mean": mean,
            "std": std,
            "maximum": maximum,
            "perc_0": perc_0,
        }
    )


results = pd.concat(
    [
        results,
        results.apply(get_availability_stats, axis=1),
    ],
    axis=1,
)


In [19]:
results

,h3_res,time_interval_length,batch_size,nodes_per_feature,n_layers,activation,dropout,train_duration,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy,median,mean,std,maximum,perc_0
0,7,1,64,1.5,3,tanh,0.1,855.901519,45.103438,6.715909,2.734877,0.450544,0.855141,0.0,5.755109,13.763612,190.0,0.624848
1,7,2,64,1.5,3,tanh,0.1,560.716685,47.166135,6.867761,2.816211,0.442670,0.853768,0.0,5.934785,14.219029,183.0,0.623708
2,7,6,64,1.5,3,tanh,0.1,236.145673,53.590551,7.320557,3.066465,0.427757,0.846844,0.0,6.711749,16.032654,174.0,0.619004
3,7,24,64,1.5,3,tanh,0.1,94.846507,93.348040,9.661679,3.883434,0.341454,0.849619,0.0,10.485084,22.668667,174.0,0.597615
4,8,1,64,1.5,3,tanh,0.1,3703.945640,5.674175,2.382053,0.983003,0.596280,0.865919,0.0,1.327515,3.423869,70.0,0.719383
5,8,2,64,1.5,3,tanh,0.1,1830.123308,5.697243,2.386890,0.961181,0.589726,0.862052,0.0,1.369063,3.520954,70.0,0.717059
6,8,6,64,1.5,3,tanh,0.1,641.114363,6.773018,2.602502,1.063654,0.559425,0.853029,0.0,1.548785,3.911882,64.0,0.707291
7,8,24,64,1.5,3,tanh,0.1,230.024005,9.372776,3.061499,1.280754,0.477692,0.889425,0.0,2.421447,5.357067,54.0,0.663272
8,9,24,64,1.5,3,tanh,0.1,498.783041,1.812260,1.346202,0.646167,0.566795,0.822044,0.0,0.727274,1.682266,30.0,0.726547
